In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
data_path = os.path.join(os.path.dirname(os.getcwd()),"data",os.listdir(os.path.join(os.path.dirname(os.getcwd()),"data"))[0])

with zipfile.ZipFile(data_path) as z:
    with z.open("sample_submission.csv") as f:
        sample_submission = pd.read_csv(f)
    with z.open("test.csv") as f:
        test = pd.read_csv(f)
    with z.open("train.csv") as f:
        train = pd.read_csv(f)

In [ ]:
train.shape,test.shape

In [ ]:
train.info()

In [ ]:
train.head(2)

In [ ]:
def get_dtypes(df):
    cols = list(df.columns)
    object_features, numerical_features = [], []
    datatypes = df.dtypes.to_dict()
    object_features = [column for column in cols if datatypes[column].name=='object']
    numerical_features = [column for column in cols if column not in object_features ]
    return object_features,numerical_features
   

In [ ]:
object_features, numerical_features = get_dtypes(train)

In [ ]:
train.describe()

In [ ]:
train.hist(bins=50, figsize=(12, 12))

In [ ]:
bins = [-np.inf,80,90,100,110,np.inf]
labels = [0,1,2,3,4]
train["hr_category"] = pd.cut(train["Heart_Rate"],
                               bins=bins,
                               labels=labels)


In [ ]:
strat_train_set, strat_test_set = train_test_split(
    train, test_size=0.2, stratify=train[[target,"IsActiveMember"]], random_state=42)

In [ ]:
sns.pairplot(train)